In [7]:
import pandas as pd
import lightgbm as lgb

In [8]:
def patients_in_line (df, row, priority=None):
            
    assessment_end_time = row['assessment_end_time']
    
    event_day = row['day']

    if not priority:
        
        value = (len(df.loc[(df['assessment_end_time']<assessment_end_time)&(df['consultation_start_time']>assessment_end_time)&(df['day']==event_day)]))
        
    if priority == 'normal':
        
        value = (len(df.loc[(df['assessment_end_time']<assessment_end_time)&(df['consultation_start_time']>assessment_end_time)&(df['day']==event_day)&(df['priority_normal']==1)]))
        
    if priority == 'urgent':
        
        value = (len(df.loc[(df['assessment_end_time']<assessment_end_time)&(df['consultation_start_time']>assessment_end_time)&(df['day']==event_day)&(df['priority_urgent']==1)]))
    
    return value

In [9]:
def make_data_set (path):

    df = pd.read_csv(path, index_col='Unnamed: 0')
    
    features = ['pain','priority']

    for feature in features:

        df = pd.concat([df.drop(feature,1), pd.get_dummies(df[feature], prefix=feature)],1)
        
    df['patient_number_in_day'] = df.groupby('day')['patient'].transform(lambda X: X - min(X) + 1)
    
    df['patients_in_line'] = df.apply(lambda X: patients_in_line(df, X),1)
    df['normal_patients_in_line'] = df.apply(lambda X: patients_in_line(df, X, priority='normal'),1)
    df['urgent_patients_in_line'] = df.apply(lambda X: patients_in_line(df, X, priority='urgent'),1)
    
    df.drop(['consultation_start_time','consultation_end_time','duration','patient','day'],1,inplace=True)
    
    return df

In [12]:
from sklearn.metrics import mean_squared_error

def rmse (targets,preds):
    
    return mean_squared_error(targets,preds)**(1/2)

In [10]:
model = lgb.Booster(model_file='model.txt') ## if not running from directory change path

In [11]:
file_path = input()

# ../data/summary.csv

df = make_data_set(file_path)

predictions = pd.np.exp(model.predict(df))

temp = pd.read_csv(file_path)

targets = temp['consultation_end_time'] - temp['assessment_end_time']

print (rmse(targets,predictions))

run cells, input path to csv summary file in the same format as exercise file. <br>
feature engineering should take a few seconds/couple minutes depending on size of dataset and hardware. <br>